# <font color='1B818D'>Star Jeans</font>
Eduardo e Marcelo são dois brasileiros, amigos e sócios de empreendimento. Depois de vários negócio bem sucedidos, eles estão planejando entrar no mercado de moda dos EUA como um modelo de negócio do tipo E-commerce.

A idéia inicial é entrar no mercado com apenas um produto e para um público específico: calças Jenas para o público masculino. O objetivo é manter o custo de operação baixo e escalar a medida que forem conseguindo clientes.

Porém, mesmo com o produto de entrada e a audiência definidos, os dois sócios não tem experiência nesse mercado de moda e portanto não sabem definir coisas básicas como preço, o tipo de calça e o material para a fabricação de cada peça.

Assim, os dois sócios contrataram uma consultoria de Ciência de Dados para responder as seguintes
perguntas: 
1. Qual o melhor preço de venda para as calças? 
2. Quantos tipos de calças e suas cores para o produto inicial? 
3. Quais as matérias-prima necessárias para confeccionar as calças?

As principais concorrentes da empresa Start Jeans são as americadas H&M e Macys.

## <font color='1B818D'>Planejamento da Solução</font>

1. Problema de Negócio
    - Qual o melhor preço de venda para as calças?
       - Mediana dos preços dos concorrentes
           - Realizar o calculo da mediana sobre o produto, tipo e cor
    - Quantos tipos de calças e suas cores para o produto inicial?
    - Quais as matérias-prima necessárias para confeccionar as calças?
<br><br>     
2. Formato de Entrega
   - Tabelas e Gráficos
        - Gráfico de barras com a mediana dos preço dos produtos, por tipo e cor dos últimos 30 dia
        - Tabela com as seguintes colunas:<br> id | product_name | product_type | product_color | product_composition | product_price
        - Definição do schema: Colunas e seu tipo
        - Definição a infraestrutura de armazenamento (SQLITE3)
        - Design do ETL (Scripts de Extração, Transformação e Carga)
        - Planejamento de Agendamento dos scripts (dependencias entre os scripts)
        - Fazer as visualizações
        - Entrega do produto final
<br><br>     
3. Local de Entrega
   - App no Streamlit
<br><br>     
4. Fonte de Dados
   - Site da H&M: https://www2.hm.com/en_us/men/products/jeans.html
   - Site da Macys: https://www.macys.com/shop/mens-clothing/mens-jeans
<br><br>
5. Ferramentas
    - Python 3.8.0
    - Bibliotecas de Webscrapping (BS4, Selenium)
    - PyCharm
    - Jupyter Notebook (análise e prototipagens)
    - Crontjob, Airflow
    - Streamlit
    - Heroku

### <font color='1B818D'>Web Scraping Showcase</font>

In [173]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from datetime import datetime
import re
import math

In [174]:
url_hm = 'https://www2.hm.com/en_us/men/products/jeans.html'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
page = requests.get(url_hm, headers=headers)

In [175]:
soup = BeautifulSoup(page.text, 'html.parser')

In [176]:
total_item = soup.find_all('h2', class_='load-more-heading')[0].get('data-total')
num_pages = math.ceil(int(total_item)/36)
num_pages

3

In [177]:
url_hm2 = url_hm + '?page-size=' + str(int(num_pages * 36))
url_hm2

'https://www2.hm.com/en_us/men/products/jeans.html?page-size=108'

In [178]:
page = requests.get(url_hm2, headers=headers)
soup = BeautifulSoup(page.text, 'html.parser')

In [179]:
products = soup.find('ul', class_='products-listing small')

In [180]:
product_list = products.find_all('article', class_='hm-product-item')

#product id
product_id = [p.get('data-articlecode') for p in product_list]

#product category
product_category = [p.get('data-category') for p in product_list]

In [181]:
product_list = products.find_all('a', class_='link')

#product name
product_name = [p.get_text() for p in product_list]

In [182]:
product_list = products.find_all('span', class_='price regular')

#product price
product_price = [p.get_text() for p in product_list]

In [183]:
data = pd.DataFrame([product_id, product_category, product_name, product_price]).T
data.columns = ['product_id', 'product_category', 'product_name', 'product_price']
data['scrapy_datetime'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
data['scrapy_datetime'] = pd.to_datetime(data['scrapy_datetime'])
data.sample(10)

,product_id,product_category,product_name,product_price,scrapy_datetime
49,0974597001,men_jeans_slim,Slim Tapered Jeans,$ 29.99,2022-03-29 21:51:23
2,0690449022,men_jeans_ripped,Skinny Jeans,$ 39.99,2022-03-29 21:51:23
32,1008549004,men_jeans_regular,Regular Jeans,$ 19.99,2022-03-29 21:51:23
77,1008110005,men_jeans_slim,Freefit® Slim Jeans,$ 49.99,2022-03-29 21:51:23
56,0811993040,men_jeans_regular,Regular Jeans,$ 29.99,2022-03-29 21:51:23
21,1004199002,men_jeans_skinny,Skinny Cropped Jeans,$ 29.99,2022-03-29 21:51:23
86,0993887002,men_jeans_joggers,Hybrid Regular Denim Joggers,$ 39.99,2022-03-29 21:51:23
61,0985197004,men_jeans_slim,Slim Jeans,$ 19.99,2022-03-29 21:51:23
14,1004199005,men_jeans_skinny,Skinny Cropped Jeans,$ 29.99,2022-03-29 21:51:23
33,1013317008,men_jeans_joggers,Hybrid Regular Tapered Joggers,$ 39.99,2022-03-29 21:51:23


### <font color='1B818D'>Web Scraping Inside Products</font>

In [184]:
url_hm = 'https://www2.hm.com/en_us/productpage.0985159001.html'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
page = requests.get(url_hm, headers=headers)

soup = BeautifulSoup(page.text, 'html.parser')

################# df_color

#color name
product_list = soup.find_all('a', class_='filter-option miniature')
product_color_name = [p.get('data-color') for p in product_list]

#product id
product_id = [p.get('data-articlecode') for p in product_list]

df_color = pd.DataFrame([product_id, product_color_name]).T
df_color.columns = ['product_id', 'product_color_name']

#creating style_id + color_id
df_color['style_id'] = df_color['product_id'].apply(lambda x: x[:-3])
df_color['color_id'] = df_color['product_id'].apply(lambda x: x[-3:])


################# df_composition

#composition
product_composition_list = soup.find_all('div', class_='details-attributes-list-item')

product_composition = [list(filter(None, p.get_text().split('\n'))) for p in product_composition_list]

df_composition = pd.DataFrame(product_composition).T
df_composition.columns = df_composition.iloc[0]

df_composition = df_composition.iloc[1:].fillna(method='ffill')
df_composition = df_composition[['Fit', 'Composition', 'Art. No.']]
df_composition = df_composition.drop_duplicates()

#creating style_id + color_id
df_composition['style_id'] = df_composition['Art. No.'].apply(lambda x: x[:-3])
df_composition['color_id'] = df_composition['Art. No.'].apply(lambda x: x[-3:])


################# merge df_color + df_composition
data_sku = pd.merge(df_color, df_composition[['Fit', 'Composition', 'style_id']], how='left', on='style_id')

In [185]:
data_sku

,product_id,product_color_name,style_id,color_id,Fit,Composition
0,0985159002,Denim blue,0985159,002,Skinny fit,"Shell: Cotton 99%, Spandex 1%"
1,0985159002,Denim blue,0985159,002,Skinny fit,Pocket lining: Cotton 100%
2,0985159003,Dark gray,0985159,003,Skinny fit,"Shell: Cotton 99%, Spandex 1%"
3,0985159003,Dark gray,0985159,003,Skinny fit,Pocket lining: Cotton 100%
4,0985159004,Light denim blue,0985159,004,Skinny fit,"Shell: Cotton 99%, Spandex 1%"
5,0985159004,Light denim blue,0985159,004,Skinny fit,Pocket lining: Cotton 100%
6,0985159005,Dark blue,0985159,005,Skinny fit,"Shell: Cotton 99%, Spandex 1%"
7,0985159005,Dark blue,0985159,005,Skinny fit,Pocket lining: Cotton 100%
8,0985159006,Dark denim blue,0985159,006,Skinny fit,"Shell: Cotton 99%, Spandex 1%"
9,0985159006,Dark denim blue,0985159,006,Skinny fit,Pocket lining: Cotton 100%


### <font color='1B818D'>Multiple Web Scraping Inside Products</font>

In [186]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

df_details = pd.DataFrame()
aux = []

cols = ['Art. No.', 'Composition', 'Fit', 'Material', 'Size',]
df_pattern = pd.DataFrame(columns=cols)

for i in range(len(data)):
    
    url_hm = 'https://www2.hm.com/en_us/productpage.' + data.loc[i, 'product_id'] + '.html'

    page = requests.get(url_hm, headers=headers)

    soup = BeautifulSoup(page.text, 'html.parser')

    ################# df_color

    #color name
    product_list = soup.find_all('a', class_='filter-option miniature')
    product_color_name = [p.get('data-color') for p in product_list]

    #product id
    product_id = [p.get('data-articlecode') for p in product_list]

    df_color = pd.DataFrame([product_id, product_color_name]).T
    df_color.columns = ['product_id', 'product_color_name']

    #creating style_id + color_id
    df_color['style_id'] = df_color['product_id'].apply(lambda x: x[:-3])
    df_color['color_id'] = df_color['product_id'].apply(lambda x: x[-3:])


    ################# df_composition

    #composition
    product_composition_list = soup.find_all('div', class_='details-attributes-list-item')

    product_composition = [list(filter(None, p.get_text().split('\n'))) for p in product_composition_list]

    df_composition = pd.DataFrame(product_composition).T
    df_composition.columns = df_composition.iloc[0]

    df_composition = df_composition.iloc[1:].fillna(method='ffill')
    
    df_composition = pd.concat([df_pattern, df_composition], axis=0)
    
    df_composition = df_composition.drop_duplicates()

    #creating style_id + color_id
    df_composition['style_id'] = df_composition['Art. No.'].apply(lambda x: x[:-3])
    df_composition['color_id'] = df_composition['Art. No.'].apply(lambda x: x[-3:])

    aux = aux + df_composition.columns.tolist()

    ################# merge df_color + df_composition
    data_sku = pd.merge(df_color, df_composition[[ 'style_id', 'Fit', 'Composition', 'Size', 'Material']], how='left', on='style_id')
    
    df_details = pd.concat([df_details, data_sku], axis=0)

In [187]:
df_details = df_details.drop_duplicates()

data['style_id'] = data['product_id'].apply(lambda x: x[:-3])

data_raw = pd.merge(data, df_details[['style_id', 'color_id', 'Fit', 'Composition', 'Material']], how='left', on='style_id')

data_raw = data_raw.drop_duplicates()

data_raw

,product_id,product_category,product_name,product_price,scrapy_datetime,style_id,color_id,Fit,Composition,Material
0,0690449043,men_jeans_ripped,Skinny Jeans,$ 39.99,2022-03-29 21:51:23,0690449,001,Skinny fit,"Cotton 98%, Spandex 2%",keyFibreType
1,0690449043,men_jeans_ripped,Skinny Jeans,$ 39.99,2022-03-29 21:51:23,0690449,001,Skinny fit,"Pocket lining: Polyester 65%, Cotton 35%",Denim
2,0690449043,men_jeans_ripped,Skinny Jeans,$ 39.99,2022-03-29 21:51:23,0690449,002,Skinny fit,"Cotton 98%, Spandex 2%",keyFibreType
3,0690449043,men_jeans_ripped,Skinny Jeans,$ 39.99,2022-03-29 21:51:23,0690449,002,Skinny fit,"Pocket lining: Polyester 65%, Cotton 35%",Denim
4,0690449043,men_jeans_ripped,Skinny Jeans,$ 39.99,2022-03-29 21:51:23,0690449,006,Skinny fit,"Cotton 98%, Spandex 2%",keyFibreType
...,...,...,...,...,...,...,...,...,...,...
2307,0993887004,men_jeans_joggers,Hybrid Regular Denim Joggers,$ 39.99,2022-03-29 21:51:23,0993887,002,Regular fit,"Cotton 77%, Polyester 21%, Spandex 2%",Denim
2308,0993887004,men_jeans_joggers,Hybrid Regular Denim Joggers,$ 39.99,2022-03-29 21:51:23,0993887,004,Regular fit,"Cotton 77%, Polyester 21%, Spandex 2%",Denim
2309,0993887004,men_jeans_joggers,Hybrid Regular Denim Joggers,$ 39.99,2022-03-29 21:51:23,0993887,001,Regular fit,"Cotton 77%, Polyester 21%, Spandex 2%",Denim
2310,0993887004,men_jeans_joggers,Hybrid Regular Denim Joggers,$ 39.99,2022-03-29 21:51:23,0993887,001,Regular fit,"Cotton 79%, Polyester 20%, Spandex 1%",Denim


In [188]:
data_raw.to_csv('data_hm.csv', index=False)

### <font color='1B818D'>Transform</font>

In [189]:
data = pd.read_csv('data_hm.csv')

data = data.dropna(subset=['color_id'])

# product_id

# product_category

# product_name
data['product_name'] = data['product_name'].apply(lambda x: x.replace(' ', '_').lower())

# product_price
data['product_price'] = data['product_price'].apply(lambda x: x.replace('$ ', '')).astype(float)

# scrapy_datetime
data['scrapy_datetime'] = pd.to_datetime(data['scrapy_datetime'], format='%Y-%m-%d %H:%M:%S')

# style_id

# color_id
data['color_id'] = data['color_id'].astype('int64')

# Fit
data['Fit'] = data['Fit'].apply(lambda x: x.replace(' ', '_').lower())

# Composition
# Material


data

,product_id,product_category,product_name,product_price,scrapy_datetime,style_id,color_id,Fit,Composition,Material
0,690449043,men_jeans_ripped,skinny_jeans,39.99,2022-03-29 21:51:23,690449,1,skinny_fit,"Cotton 98%, Spandex 2%",keyFibreType
1,690449043,men_jeans_ripped,skinny_jeans,39.99,2022-03-29 21:51:23,690449,1,skinny_fit,"Pocket lining: Polyester 65%, Cotton 35%",Denim
2,690449043,men_jeans_ripped,skinny_jeans,39.99,2022-03-29 21:51:23,690449,2,skinny_fit,"Cotton 98%, Spandex 2%",keyFibreType
3,690449043,men_jeans_ripped,skinny_jeans,39.99,2022-03-29 21:51:23,690449,2,skinny_fit,"Pocket lining: Polyester 65%, Cotton 35%",Denim
4,690449043,men_jeans_ripped,skinny_jeans,39.99,2022-03-29 21:51:23,690449,6,skinny_fit,"Cotton 98%, Spandex 2%",keyFibreType
...,...,...,...,...,...,...,...,...,...,...
2307,993887004,men_jeans_joggers,hybrid_regular_denim_joggers,39.99,2022-03-29 21:51:23,993887,2,regular_fit,"Cotton 77%, Polyester 21%, Spandex 2%",Denim
2308,993887004,men_jeans_joggers,hybrid_regular_denim_joggers,39.99,2022-03-29 21:51:23,993887,4,regular_fit,"Cotton 77%, Polyester 21%, Spandex 2%",Denim
2309,993887004,men_jeans_joggers,hybrid_regular_denim_joggers,39.99,2022-03-29 21:51:23,993887,1,regular_fit,"Cotton 77%, Polyester 21%, Spandex 2%",Denim
2310,993887004,men_jeans_joggers,hybrid_regular_denim_joggers,39.99,2022-03-29 21:51:23,993887,1,regular_fit,"Cotton 79%, Polyester 20%, Spandex 1%",Denim


In [190]:
data.dtypes

product_id                   int64
product_category            object
product_name                object
product_price              float64
scrapy_datetime     datetime64[ns]
style_id                     int64
color_id                     int64
Fit                         object
Composition                 object
Material                    object
dtype: object

In [191]:
data.isna().sum()

product_id          0
product_category    0
product_name        0
product_price       0
scrapy_datetime     0
style_id            0
color_id            0
Fit                 0
Composition         0
Material            0
dtype: int64

In [192]:
text01 = 'Rua Armando Shibata, 97 - Vila Leopolding, São Paulo'
text02 = 'Av. dos trabalhadores, 403 - Centro, Mogi Guaçu'
text03 = 'Al. Moraes, 90 - Vila Golmes, Pernambuco'

text = [text01, text02, text03]

regex = '^[^,]*'

In [193]:
for t in text:
    print(re.search(regex, t).group(0))

Rua Armando Shibata
Av. dos trabalhadores
Al. Moraes


In [194]:
regex = '\d+'

for t in text:
    print(re.search(regex, t).group(0))

97
403
90


In [195]:
regex = '- (.+),'

for t in text:
    print(re.search(regex, t).group(1))

Vila Leopolding
Centro
Vila Golmes


In [196]:
regex = ', ([a-zA-Z].+)'

for t in text:
    print(re.search(regex, t).group(1))

São Paulo
Mogi Guaçu
Pernambuco
